# HW#6 Regularization

안녕하세요, 광운대학교 로봇학부의 오정현 교수입니다. 본 자료는 딥러닝 실습 수업을 위해 제작된 것입니다.

파이썬 문법
- 점프투파이썬(https://wikidocs.net/book/1) 참고

이번 과제는 딥러닝의 일반화 성능을 높이기 위한 Regularization을 해보는 것입니다. CNN을 이용한 이미지 분류에 여러 가지 Regularization 기법을 적용해 보도록 하겠습니다. 대표적인 Regularization 기법으로 Dropout, Data augmentation, Batch Normalization 등이 있습니다.

#1. Data Generation
Data는 mnist dataset을 이용하도록 하겠습니다. mnist dataset은 원래 60000개의 training set이 주어져 있지만 overfitting을 유도하기 위하여 1000개의 data만 이용하려고 합니다. 1000개의 data로 이루어진 x_train과 y_train을 만들어 보세요. 그리고 2000개로 이루어진 large_x_train, large_y_train을 만들어보세요.

In [2]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
import tensorflow as tf

batch_size = 28
num_classes = 10
epochs = 50

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### START CODE HERE ###
large_x_train = x_train[1000:3000]
large_y_train= y_train[1000:3000]
x_train = x_train[:1000]
y_train = y_train[:1000]
### END CODE HERE ###

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    large_x_train = large_x_train.reshape(large_x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    large_x_train = large_x_train.reshape(large_x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
large_x_train = large_x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
large_x_train /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
large_y_train = keras.utils.to_categorical(large_y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

assert large_x_train.shape[0]==2000
assert large_y_train.shape[0]==2000
assert x_train.shape[0]==1000
assert y_train.shape[0]==1000
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("large_x_train shape:", large_x_train.shape)
print("large_y_train shape:", large_y_train.shape)

x_train shape: (1000, 28, 28, 1)
y_train shape: (1000, 10)
large_x_train shape: (2000, 28, 28, 1)
large_y_train shape: (2000, 10)


#2. CNN Model
복습 차원에서 기본 CNN Model을 만들어 보도록 하겠습니다. 다음과 같은 CNN 모델을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Conv2D | (None, 26, 26, 32) | 320 |
| Activation | (None, 26, 26, 32) | 0 |
| MaxPooling2D | (None, 13, 13, 32) | 0 |
| Conv2D | (None, 11, 11, 32) | 9248 |
| Activation | (None, 11, 11, 32) | 0 |
| MaxPooling2D | (None, 5, 5, 32) | 0 |
| Flatten | (None, 800) | 0 |
| Dense | (None, 10) | 8010 |


In [2]:

### START CODE HERE ###
model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (28,28,1)),
                                    #activation 포함 (relu)
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (28,28,1)),
                                    #activation 포함 (relu)
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(10,activation='softmax')
                                    #마지막 dense에 acitvation은 softmax

])
### END CODE HERE ###

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 800)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                8010      
Total params: 17,578
Trainable params: 17,578
Non-trainable params: 0
____________________________________________________

#3. Learning CNN
기본 CNN Model을 학습해 보겠습니다. Overfitting은 Training data에 맞추어 과도하게 학습이 이루어져 Test data에서 높은 성능이 나지 않는 현상, 즉 Generalization 성능이 높지 않게 나타나는 현상을 의미합니다. 따라서 Overfitting이 발생하면 Training accuracy는 높지만 Test accuracy는 높지 않게 나타납니다. 

현재 모델이 overfitting이 발생하는지 체크해 보세요. 일부러 overfitting이 발생하도록 유도하였기 때문에 overfitting 현상이 나타나야 합니다.

In [3]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])
weights = model.get_weights()

history=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/50
36/36 [==============================] - 3s 88ms/step - loss: 2.3705 - accuracy: 0.1160 - val_loss: 2.3698 - val_accuracy: 0.1116
Epoch 2/50
36/36 [==============================] - 3s 84ms/step - loss: 2.3693 - accuracy: 0.1160 - val_loss: 2.3687 - val_accuracy: 0.1122
Epoch 3/50
36/36 [==============================] - 3s 83ms/step - loss: 2.3682 - accuracy: 0.1160 - val_loss: 2.3676 - val_accuracy: 0.1124
Epoch 4/50
36/36 [==============================] - 3s 83ms/step - loss: 2.3671 - accuracy: 0.1170 - val_loss: 2.3666 - val_accuracy: 0.1127
Epoch 5/50
36/36 [==============================] - 3s 83ms/step - loss: 2.3659 - accuracy: 0.1180 - val_loss: 2.3655 - val_accuracy: 0.1127
Epoch 6/50
36/36 [==============================] - 3s 83ms/step - loss: 2.3648 - accuracy: 0.1180 - val_loss: 2.3644 - val_accuracy: 0.1128
Epoch 7/50
36/36 [==============================] - 3s 83ms/step - loss: 2.3637 - accuracy: 0.1180 - val_loss: 2.3633 - val_accuracy: 0.1132
Epoch 8/50
36

#4. Regularization
Overfitting이 발생한 모델에 다양한 Regularization 기법을 이용해 보도록 합시다.

4.1 Large Dataset

Training data가 충분하다면 overfitting 현상이 발생할 가능성이 줄어듭니다. 기본 CNN 분류 모델에서 large_x_train과 large_y_train을 이용하면 성능이 올라갈 것입니다. Generalization 성능이 올라갔나요?

In [4]:
#initialize weights
model.set_weights(weights)

large_model_history=model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Largemodel Test loss:', score[0])
print('Largemodel Test accuracy:', score[1])

Epoch 1/50
72/72 [==============================] - 4s 54ms/step - loss: 2.3745 - accuracy: 0.1105 - val_loss: 2.3681 - val_accuracy: 0.1126
Epoch 2/50
72/72 [==============================] - 4s 55ms/step - loss: 2.3715 - accuracy: 0.1120 - val_loss: 2.3654 - val_accuracy: 0.1130
Epoch 3/50
72/72 [==============================] - 4s 54ms/step - loss: 2.3687 - accuracy: 0.1130 - val_loss: 2.3627 - val_accuracy: 0.1135
Epoch 4/50
72/72 [==============================] - 4s 55ms/step - loss: 2.3659 - accuracy: 0.1135 - val_loss: 2.3601 - val_accuracy: 0.1148
Epoch 5/50
72/72 [==============================] - 4s 55ms/step - loss: 2.3633 - accuracy: 0.1145 - val_loss: 2.3576 - val_accuracy: 0.1149
Epoch 6/50
72/72 [==============================] - 4s 55ms/step - loss: 2.3607 - accuracy: 0.1145 - val_loss: 2.3551 - val_accuracy: 0.1158
Epoch 7/50
72/72 [==============================] - 4s 55ms/step - loss: 2.3581 - accuracy: 0.1160 - val_loss: 2.3527 - val_accuracy: 0.1161
Epoch 8/50
72

##4.2 Dropout
Dropout은 쉽게 쓸 수 있는 Regularization 기법입니다. Layer 사이에 Dropout layer만 추가하면 되기 때문에 간편합니다. 다음과 같은 Dropout model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Conv2D | (None, 26, 26, 32) | 320 |
| Activation | (None, 26, 26, 32) | 0 |
| MaxPooling2D | (None, 13, 13, 32) | 0 |
| Conv2D | (None, 11, 11, 32) | 9248 |
| Activation | (None, 11, 11, 32) | 0 |
| MaxPooling2D | (None, 5, 5, 32) | 0 |
| Dropout | (None, 5, 5, 32) | 0 |
| Flatten | (None, 800) | 0 |
| Dropout | (None, 800) | 0 |
| Dense | (None, 10) | 8010 |

In [5]:

### START CODE HERE ###
dropout_model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (28,28,1)),
                                    #activation 포함 (relu)
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (28,28,1)),
                                    #activation 포함 (relu)
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                    tf.keras.layers.Dropout(0.2),

                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(10,activation='softmax')
                                    #마지막 dense에 acitvation은 softmax

])
### END CODE HERE ###

dropout_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 800)              

Dropout Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

In [12]:
dropout_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

dropout_model_history=dropout_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = dropout_model.evaluate(x_test, y_test, verbose=0)
print('Dropout model Test loss:', score[0])
print('Dropout model Test accuracy:', score[1])

Epoch 1/50
36/36 [==============================] - 3s 93ms/step - loss: 2.2929 - accuracy: 0.1200 - val_loss: 2.2938 - val_accuracy: 0.1071
Epoch 2/50
36/36 [==============================] - 3s 87ms/step - loss: 2.2919 - accuracy: 0.1360 - val_loss: 2.2933 - val_accuracy: 0.1081
Epoch 3/50
36/36 [==============================] - 3s 88ms/step - loss: 2.2928 - accuracy: 0.1320 - val_loss: 2.2928 - val_accuracy: 0.1096
Epoch 4/50
36/36 [==============================] - 3s 88ms/step - loss: 2.2867 - accuracy: 0.1420 - val_loss: 2.2923 - val_accuracy: 0.1108
Epoch 5/50
36/36 [==============================] - 3s 86ms/step - loss: 2.2918 - accuracy: 0.1340 - val_loss: 2.2918 - val_accuracy: 0.1117
Epoch 6/50
36/36 [==============================] - 3s 87ms/step - loss: 2.2898 - accuracy: 0.1390 - val_loss: 2.2913 - val_accuracy: 0.1127
Epoch 7/50
36/36 [==============================] - 3s 88ms/step - loss: 2.2843 - accuracy: 0.1410 - val_loss: 2.2908 - val_accuracy: 0.1143
Epoch 8/50
36

In [49]:

### START CODE HERE ###
dropout_model_50 = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (28,28,1)),
                                    #activation 포함 (relu)
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (28,28,1)),
                                    #activation 포함 (relu)
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                    tf.keras.layers.Dropout(0.5),

                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dropout(0.5),
                                    tf.keras.layers.Dense(10,activation='softmax')
                                    #마지막 dense에 acitvation은 softmax

])
### END CODE HERE ###

dropout_model_50.summary()

dropout_model_50.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

dropout_model_50_history=dropout_model_50.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = dropout_model_50.evaluate(x_test, y_test, verbose=0)
print('Dropout 0.8 model Test loss:', score[0])
print('Dropout 0.8model Test accuracy:', score[1])


Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_28 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 11, 11, 32)        9248      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 800)               0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 800)             

##4.3 BatchNormalization
BatchNormalization(BN)은 쉽게 쓸 수 있는 Regularization 기법입니다. Layer 사이에 BN layer만 추가하면 되기 때문에 간편합니다. 다음과 같은 BN model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Conv2D | (None, 26, 26, 32) | 320 |
| BatchNormalization | (None, 26, 26, 32) | 128 |
| Activation | (None, 26, 26, 32) | 0 |
| MaxPooling2D | (None, 13, 13, 32) | 0 |
| Conv2D | (None, 11, 11, 32) | 9248 |
| BatchNormalization | (None, 11, 11, 32) | 128 |
| Activation | (None, 11, 11, 32) | 0 |
| MaxPooling2D | (None, 5, 5, 32) | 0 |
| Flatten | (None, 800) | 0 |
| Dense | (None, 10) | 8010 |

In [21]:

### START CODE HERE ###
bn_model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (28,28,1)),
                                    #activation 포함 (relu)
                                    tf.keras.layers.BatchNormalization(), 
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                    
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (28,28,1)),
                                    #activation 포함 (relu)
                                    tf.keras.layers.BatchNormalization(), 
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),

                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(10,activation='softmax')
                                    #마지막 dense에 acitvation은 softmax

])
### END CODE HERE ###
bn_model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_6 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 11, 11, 32)        9248      
_________________________________________________________________
batch_normalization_7 (Batch (None, 11, 11, 32)        128       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 800)             

BN Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

In [22]:
bn_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

bn_model_history=bn_model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = bn_model.evaluate(x_test, y_test, verbose=0)
print('BN model Test loss:', score[0])
print('BN model Test accuracy:', score[1])

Epoch 1/50
36/36 [==============================] - 4s 109ms/step - loss: 4.4643 - accuracy: 0.0670 - val_loss: 2.3100 - val_accuracy: 0.1075
Epoch 2/50
36/36 [==============================] - 4s 104ms/step - loss: 4.4040 - accuracy: 0.0670 - val_loss: 2.3255 - val_accuracy: 0.1014
Epoch 3/50
36/36 [==============================] - 4s 104ms/step - loss: 4.3541 - accuracy: 0.0660 - val_loss: 2.3469 - val_accuracy: 0.1015
Epoch 4/50
36/36 [==============================] - 4s 103ms/step - loss: 4.2981 - accuracy: 0.0690 - val_loss: 2.3702 - val_accuracy: 0.1030
Epoch 5/50
36/36 [==============================] - 4s 103ms/step - loss: 4.2509 - accuracy: 0.0700 - val_loss: 2.3943 - val_accuracy: 0.1036
Epoch 6/50
36/36 [==============================] - 4s 103ms/step - loss: 4.2008 - accuracy: 0.0710 - val_loss: 2.4198 - val_accuracy: 0.1050
Epoch 7/50
36/36 [==============================] - 4s 103ms/step - loss: 4.1511 - accuracy: 0.0740 - val_loss: 2.4486 - val_accuracy: 0.1082
Epoch 

##4.4 Final Model
지금까지 썼던 Regularization 기법들을 종합선물세트로 적용해 봅시다. 다음과 같은 Final model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Conv2D | (None, 26, 26, 32) | 320 |
| BatchNormalization | (None, 26, 26, 32) | 128 |
| Activation | (None, 26, 26, 32) | 0 |
| MaxPooling2D | (None, 13, 13, 32) | 0 |
| Conv2D | (None, 11, 11, 32) | 9248 |
| BatchNormalization | (None, 11, 11, 32) | 128 |
| Activation | (None, 11, 11, 32) | 0 |
| MaxPooling2D | (None, 5, 5, 32) | 0 |
| Dropout | (None, 5, 5, 32) | 0 |
| Flatten | (None, 800) | 0 |
| Dropout | (None, 800) | 0 |
| Dense | (None, 10) | 8010 |

In [3]:

### START CODE HERE ###
final_model = tf.keras.models.Sequential([
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (28,28,1)),
                                    #activation 포함 (relu)
                                    tf.keras.layers.BatchNormalization(), 
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                    
                                    
                                    tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape = (28,28,1)),
                                    #activation 포함 (relu)
                                    tf.keras.layers.BatchNormalization(), 
                                    tf.keras.layers.MaxPool2D(pool_size=(2,2)),
                                    tf.keras.layers.Dropout(0.2),


                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dropout(0.2),
                                    tf.keras.layers.Dense(10,activation='softmax')
                                    #마지막 dense에 acitvation은 softmax

])
### END CODE HERE ###
final_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
batch_normalization_1 (Batch (None, 11, 11, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
dropout (Dropout)            (None, 5, 5, 32)          0

Final Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

In [24]:
final_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

final_model_history=final_model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = final_model.evaluate(x_test, y_test, verbose=0)
print('Final model Test loss:', score[0])
print('Final model Test accuracy:', score[1])

Epoch 1/50
72/72 [==============================] - 5s 68ms/step - loss: 4.3188 - accuracy: 0.1240 - val_loss: 2.3257 - val_accuracy: 0.0988
Epoch 2/50
72/72 [==============================] - 5s 68ms/step - loss: 4.1781 - accuracy: 0.1230 - val_loss: 2.3565 - val_accuracy: 0.1035
Epoch 3/50
72/72 [==============================] - 5s 70ms/step - loss: 4.0779 - accuracy: 0.1295 - val_loss: 2.3734 - val_accuracy: 0.0988
Epoch 4/50
72/72 [==============================] - 5s 69ms/step - loss: 4.0925 - accuracy: 0.1250 - val_loss: 2.3879 - val_accuracy: 0.1078
Epoch 5/50
72/72 [==============================] - 5s 67ms/step - loss: 4.1400 - accuracy: 0.1215 - val_loss: 2.4241 - val_accuracy: 0.1221
Epoch 6/50
72/72 [==============================] - 5s 67ms/step - loss: 4.0019 - accuracy: 0.1265 - val_loss: 2.5001 - val_accuracy: 0.1352
Epoch 7/50
72/72 [==============================] - 5s 68ms/step - loss: 4.0002 - accuracy: 0.1405 - val_loss: 2.6154 - val_accuracy: 0.1431
Epoch 8/50
72

(optional) Training data가 늘어나면 regularization 효과가 나는 것을 보였습니다. 쉽게 training data를 늘릴 수 있는 방법은 data augmentation 입니다. 이 방법은 기존 training data를 적절히 rotating, flipping, scaling, shifting 하여 training data 수를 늘리는 것입니다. data augmentation의 regularization 효과를 테스트해 보세요.

In [0]:
datagen  = tf.keras.preprocessing.image.ImageDataGenerator(
		rotation_range=20,
		zoom_range=0.15,
		width_shift_range=0.2,
		height_shift_range=0.2,
		shear_range=0.15,
		horizontal_flip=True,
		fill_mode="nearest")

datagen.fit(x_train)


In [30]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [50]:
history_datagen =final_model.fit(datagen.flow(large_x_train, large_y_train, batch_size=batch_size),
          steps_per_epoch=len(large_x_train) / batch_size, epochs=epochs)

score = final_model.evaluate(x_test, y_test, verbose=0)
print('Final model Test loss:', score[0])
print('Final model Test accuracy:', score[1])

Epoch 1/50
72/71 [==============================] - 3s 36ms/step - loss: 3.5734 - accuracy: 0.1240
Epoch 2/50
72/71 [==============================] - 3s 36ms/step - loss: 3.5371 - accuracy: 0.1385
Epoch 3/50
72/71 [==============================] - 3s 35ms/step - loss: 3.4831 - accuracy: 0.1410
Epoch 4/50
72/71 [==============================] - 3s 36ms/step - loss: 3.5141 - accuracy: 0.1365
Epoch 5/50
72/71 [==============================] - 3s 35ms/step - loss: 3.5394 - accuracy: 0.1385
Epoch 6/50
72/71 [==============================] - 3s 35ms/step - loss: 3.6497 - accuracy: 0.1225
Epoch 7/50
72/71 [==============================] - 3s 36ms/step - loss: 3.5548 - accuracy: 0.1365
Epoch 8/50
72/71 [==============================] - 3s 36ms/step - loss: 3.5525 - accuracy: 0.1180
Epoch 9/50
72/71 [==============================] - 3s 36ms/step - loss: 3.6124 - accuracy: 0.1305
Epoch 10/50
72/71 [==============================] - 3s 36ms/step - loss: 3.5743 - accuracy: 0.1255
Epoch 11/

In [7]:
final_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

for e in range(100):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(large_x_train, large_y_train, batch_size=batch_size):
        final_model.fit(x_batch, y_batch)
        batches += 1
        if batches >= len(large_x_train) / batch_size:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
1/1 [==============================] - 0s 824us/step - loss: 4.9018 - accuracy: 0.0000e+00
Epoch 32
1/1 [==============================] - 0s 842us/step - loss: 3.2268 - accuracy: 0.1667
Epoch 33
1/1 [==============================] - 0s 932us/step - loss: 4.2610 - accuracy: 0.0833
Epoch 34
1/1 [==============================] - 0s 848us/step - loss: 3.3772 - accuracy: 0.1667
Epoch 35
1/1 [==============================] - 0s 873us/step - loss: 3.1626 - accuracy: 0.0833
Epoch 36
1/1 [==============================] - 0s 789us/step - loss: 4.2959 - accuracy: 0.0833
Epoch 37
1/1 [==============================] - 0s 759us/step - loss: 4.9672 - accuracy: 0.0833
Epoch 38
1/1 [==============================] - 0s 877us/step - loss: 3.9227 - accuracy: 0.1667
Epoch 39
1/1 [==============================] - 0s 757us/step - loss: 3.1396 - accuracy: 0.1667
Epoch 40
1/1 [==============================] - 0s 850us/step - loss: 3.5182 - accuracy: 0.0000e+00
Epoch

In [9]:
score = final_model.evaluate(x_test, y_test, verbose=0)
print('Final model Test loss:', score[0])
print('Final model Test accuracy:', score[1])

Final model Test loss: 2.762951374053955
Final model Test accuracy: 0.18649999797344208
